A simple angular 2D matching algorithm for calculating one-to-one (and multiple) matches between galaxy clusters in different cluster catalogs. It takes the center of the first cluster as a reference, and looks for cluster centers (in the other catalog) within a certain distance of the center of the reference cluster. It is trivial to add a redshift cut to the matching procedure, if the redshifts for both cluster catalogs are available. 

In [1]:
#Import pandas and numpy

import pandas as pd
import numpy as np
pd.options.display.max_rows = 50
pd.options.display.max_columns = 50

In [2]:
#Import the AMF DR9 and Planck SZ2 catalogs that we're using for comparison here

df_amfmain = pd.read_csv('dr9_main.csv',header=-1)
df_plsz2 = pd.read_csv('planck_sz2.csv',header=-1)

In [3]:
df_amfmain.columns = ['amf_no','amf_ra','amf_dec','amf_z','amf_lk','amf_rh','amf_r200',\
              'amf_rc','amf_conc','amf_edge','amf_str_rh']

df_plsz2.columns = ['pl_no','pl_name','pl_glon','pl_glat','pl_ra','pl_dec','pl_poserr','pl_sn',\
                   'pl_pipe','pl_pipedet','pl_pccs2','pl_psz','pl_irflag','pl_qneural','pl_y5r500',\
                   'pl_y5r500err','pl_valid','pl_zid','pl_z','pl_msz','pl_mszerrup','pl_mszerrlow',\
                   'pl_mcxc','pl_red','pl_act','pl_spt','pl_wise','pl_ami','pl_cosmo','pl_comment']

In [4]:
df_amfmain.head()

,amf_no,amf_ra,amf_dec,amf_z,amf_lk,amf_rh,amf_r200,amf_rc,amf_conc,amf_edge,amf_str_rh
0,4,23.9128,20.7465,0.0601,176.9125,219.3663,1.946,0.925,2.104,1.0,73.0001
1,5,260.6324,32.1398,0.2252,222.1219,200.7117,1.784,0.625,2.856,1.0,35.0001
2,6,197.8796,-1.3356,0.2042,171.6872,192.0829,1.773,0.227,7.792,1.0,9.0001
3,7,250.1485,46.6917,0.2248,225.9874,182.3373,1.725,0.495,3.483,1.0,36.0001
4,8,346.3402,21.0378,0.1453,121.0124,182.2769,1.775,0.440,4.034,1.0,72.0001


In [5]:
df_plsz2.head()

,pl_no,pl_name,pl_glon,pl_glat,pl_ra,pl_dec,pl_poserr,pl_sn,pl_pipe,pl_pipedet,pl_pccs2,pl_psz,pl_irflag,pl_qneural,pl_y5r500,pl_y5r500err,pl_valid,pl_zid,pl_z,pl_msz,pl_mszerrup,pl_mszerrlow,pl_mcxc,pl_red,pl_act,pl_spt,pl_wise,pl_ami,pl_cosmo,pl_comment
0,1,PSZ2 G000.04+45.13,0.040543,45.135175,229.190512,-1.017222,4.107310,6.753186,2,111,False,1,0,0.938825,5.481591,1.899500,20,RXC J1516.5-0056,0.1198,3.962411,0.393290,0.370242,J1516.5-0056,RMJ151653.9-010506.3,NaN,NaN,-10,-1000.0,True,NaN
1,2,PSZ2 G000.13+78.04,0.138058,78.042114,203.558683,20.255990,2.056201,9.256691,2,111,True,1227,0,0.939320,4.360848,1.840251,20,RXC J1334.1+2013,0.1710,5.122391,0.351061,0.322839,J1334.1+2013,RMJ133408.7+201453.0,NaN,NaN,-10,-100.0,True,"Point sources at: 353GHz,"
2,3,PSZ2 G000.40-41.86,0.402995,-41.860793,316.084485,-41.354169,2.427383,9.704281,1,111,False,2,0,0.988699,4.507689,1.046871,21,RXCJ2104.3-4120,0.1651,5.297053,0.320350,0.344108,J2104.3-4120,NaN,NaN,NaN,-10,-1000.0,True,NaN
3,4,PSZ2 G000.77-35.69,0.775050,-35.699386,307.972844,-40.598725,2.343365,6.581792,2,111,False,3,0,0.985064,1.606287,0.516778,21,RXCJ2031.8-4037,0.3416,6.333562,0.589880,0.611330,J2031.8-4037,NaN,NaN,SPT-CLJ2031-4037,2,-1000.0,True,NaN
4,5,PSZ2 G002.04-22.15,2.045799,-22.152166,291.359610,-36.517944,5.020757,5.125627,3,1,False,-1,0,0.399731,1.927779,0.644426,-1,NaN,-1.0000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-3,-1000.0,False,NaN


In [7]:
#Read in the table of redshifts and comoving distances 
#Can be calculated from http://www.astro.ucla.edu/~wright/CosmoCalc.html

df_discom = pd.read_csv('discom.csv',header=-1)
df_discom.columns = ['dcomi','dcomo']
df_discom.head()

,dcomi,dcomo
0,0.0000,0.0000
1,0.0001,0.2940
2,0.0002,0.5940
3,0.0003,0.8939
4,0.0004,1.1939


In [8]:
#Store data in numpy arrays

dcomz = np.array(df_discom['dcomi'])
dcomdist = np.array(df_discom['dcomo'])

amf_no = np.array(df_amfmain['amf_no'])
amf_ra = np.array(df_amfmain['amf_ra'])
amf_dec = np.array(df_amfmain['amf_dec'])
amf_z = np.array(df_amfmain['amf_z'])
amf_rh = np.array(df_amfmain['amf_rh'])

pl_no = np.array(df_plsz2['pl_no'])
pl_ra = np.array(df_plsz2['pl_ra'])
pl_dec = np.array(df_plsz2['pl_dec'])
pl_sn = np.array(df_plsz2['pl_sn'])
pl_z = np.array(df_plsz2['pl_z'])
pl_msz = np.array(df_plsz2['pl_msz'])

In [10]:
#Comoving Distance

def dcom(zco,a,b):
    ii = int(zco*10000.0+1)
    if ((ii>=1) & (ii<10000)):
        return(b[ii]+((zco-a[ii])/(a[ii+1]-a[ii]))*(b[ii+1]-b[ii]))
    elif (ii==10001):
        return(b[ii])
    else:
        print('! error for dcom : z is out of the range [0.0,1.0]')
        
# These are to save computing cycles - do a linear interpolation rather than
# a calculation with a transcendental function. There's an expected range
# that goes with actual physical galaxies. 

#Note:: This was designed to speed up the original program (written in FORTRAN 90). I plan to rewrite the code 
# for Python when I have the time. 
        

In [11]:
pi = np.pi

In [12]:
s1 = pd.Series()
s2 = pd.Series()
s3 = pd.Series()
s4 = pd.Series()
s5 = pd.Series()
s6 = pd.Series()
s7 = pd.Series()
s8 = pd.Series()
s9 = pd.Series()
s10 = pd.Series()
s11 = pd.Series()

for i in range(0,46479):
    r200ang = 3.5 * (1.0 + amf_z[i])/dcom(amf_z[i],dcomz,dcomdist)
    
    for j in range(0,1653):
        
        angs=np.arccos(np.sin(pi*amf_dec[i]/180.0)*np.sin(pi*pl_dec[j]/180.0) +
                np.cos(pi*amf_dec[i]/180.0)*np.cos(pi*pl_dec[j]/180.0)*np.cos(pi*(amf_ra[i]-pl_ra[j])/180.0))
        
        if angs < r200ang:
                  s1 = s1.append(pd.Series(amf_no[i]))
                  s2 = s2.append(pd.Series(pl_no[j]))
                  s3 = s3.append(pd.Series(amf_ra[i]))
                  s4 = s4.append(pd.Series(amf_dec[i]))
                  s5 = s5.append(pd.Series(pl_ra[j]))
                  s6 = s6.append(pd.Series(pl_dec[j]))  
                  s7 = s7.append(pd.Series(amf_z[i]))
                  s8 = s8.append(pd.Series(pl_z[j]))
                  s9 = s9.append(pd.Series(amf_rh[i]))
                  s10 = s10.append(pd.Series(pl_sn[j]))
                  s11 = s11.append(pd.Series(pl_msz[j])) 

In [13]:
#Probably not an efficient way of doing this, but it works if there no time/computational constraints. 
#I had to run the above code a couple of times, and each run took about 20 minutes on my i5 2.4GHz Mac 8 GB memory. 

In [36]:
s1.index = np.arange(0,1434)
s2.index = np.arange(0,1434)
s3.index = np.arange(0,1434)
s4.index = np.arange(0,1434)
s5.index = np.arange(0,1434)
s6.index = np.arange(0,1434)
s7.index = np.arange(0,1434)
s8.index = np.arange(0,1434)
s9.index = np.arange(0,1434)
s10.index = np.arange(0,1434)
s11.index = np.arange(0,1434)

# 1434 is the number of clusters matched between the 2 catalogs (with multiple matches). This is definitely lazy. Will
# improve later

TypeError: list indices must be integers or slices, not str